# LDA Modelling - 3rd Run

This model uses fewer types of words, restricting word types to either:

1. n-grams and nouns only, or
2. n-grams, nouns, and verbs (newly added during deployment)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string
import nltk
from nltk import WordNetLemmatizer
import spacy
import re
import bbcode
import json

import gensim
from gensim.test.utils import datapath
from gensim import corpora, models, similarities
from gensim.models import CoherenceModel, LdaModel, LdaMulticore
from gensim.parsing.preprocessing import STOPWORDS

import pyLDAvis
import pyLDAvis.gensim as p_gensim

import os
import pathlib
%matplotlib inline

In [2]:
final_df = pd.read_csv('../supercase-airplane/data/data_final.csv',index_col=0)

In [3]:
final_df = final_df[['review_comment','2gram_reviews','3gram_reviews']]
final_df.head()

,review_comment,2gram_reviews,3gram_reviews
0,"['Greensboro', 'to', 'Las', 'Vegas', 'via', 'C...","['Greensboro', 'Las_Vegas', 'via_Charlotte', '...","['Greensboro', 'Las_Vegas_via_Charlotte', 'I_w..."
1,"['Not', 'Verified', 'Tokyo', 'to', 'Dallas', '...","['Not_Verified', 'Tokyo', 'Dallas', 'dread', '...","['Not_Verified', 'Tokyo', 'Dallas', 'dread', '..."
2,"['Providence', 'to', 'Tucson', 'via', 'Chicago...","['Providence', 'Tucson', 'via_Chicago', 'Ameri...","['Providence', 'Tucson', 'via_Chicago', 'Ameri..."
3,"['Not', 'Verified', 'Philadelphia', 'to', 'Bos...","['Not_Verified', 'Philadelphia', 'Boston', 'Ho...","['Not_Verified_Philadelphia', 'Boston', 'Horri..."
4,"['Houston', 'to', 'Dallas', 'I', 'had', 'depar...","['Houston', 'Dallas', 'depart', 'American_Airl...","['Houston', 'Dallas', 'depart', 'American_Airl..."


In [4]:
final_df['review_comment'] = final_df['review_comment'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) #n-grams underscores must be preserved for readability
final_df['2gram_reviews'] = final_df['2gram_reviews'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) 
final_df['3gram_reviews'] = final_df['3gram_reviews'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) 
#Reading in the DF from a CSV turned the list of words in each cell into string, so we have to remove the punctuation and split them again to get lists of terms

In [5]:
final_df.head()

,review_comment,2gram_reviews,3gram_reviews
0,"[Greensboro, to, Las, Vegas, via, Charlotte, I...","[Greensboro, Las_Vegas, via_Charlotte, will_ne...","[Greensboro, Las_Vegas_via_Charlotte, I_will_n..."
1,"[Not, Verified, Tokyo, to, Dallas, I, was, dre...","[Not_Verified, Tokyo, Dallas, dread, flight, b...","[Not_Verified, Tokyo, Dallas, dread, flight, b..."
2,"[Providence, to, Tucson, via, Chicago, America...","[Providence, Tucson, via_Chicago, American_Air...","[Providence, Tucson, via_Chicago, American_Air..."
3,"[Not, Verified, Philadelphia, to, Boston, Horr...","[Not_Verified, Philadelphia, Boston, Horrible_...","[Not_Verified_Philadelphia, Boston, Horrible_c..."
4,"[Houston, to, Dallas, I, had, departed, from, ...","[Houston, Dallas, depart, American_Airlines, t...","[Houston, Dallas, depart, American_Airlines, t..."


In [6]:
final_df.dtypes

review_comment    object
2gram_reviews     object
3gram_reviews     object
dtype: object

In [45]:
# Text Cleaning Redux

nlp = spacy.load("en_core_web_sm")
parser = bbcode.Parser()

#expand contractions
with open('./en_contractions/contra_dict.txt') as contra_dict:
    cList = json.load(contra_dict)

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

#convert numbers to words

num_dict = {'0':'zero',
            '1':'one',
            '2':'two',
            '3':'three',
            #'i':'one',      skipped. Using Roman numeral 'i' will conflict with the pronoun "I", which is not a number
            'ii':'two',
            'iii':'three'   
            }

def num2word(d):
    
    if (len(d) == 1 and d in '0123')  or (d in ['ii','iii']):
        word = num_dict[d]
    
    elif (len(str(d))==1 and str(d) in '0123'):
        word = num_dict(str(d))
    
    else:
        word = d
    
    return word

#define stopwords

en_stopwords = list(set(STOPWORDS))
en_stopwords.extend(['good','better','great','lot','game','like','I','i'])
en_stopwords = [w for w in en_stopwords if w not in ['one','two','three']]    #retain these for making n-grams, then remove afterwards

def remove_stopwords(doc):
    words = [num2word(w) for w in doc if w != '' and w not in en_stopwords]
    return words


#combine cleaning functions into one function
def parse_clean(text):
    parsed_text = parser.strip(text) #remove BBcode notations from text
    
    text = expandContractions(parsed_text) #expand contractions; return all text in lower case
    
    text = re.split(r'\W+',text) #separate words from punctuation (e.g. remove "'s" from "Cao Cao's")
    
    text = [num2word(w) for w in text] #convert single digits to words before word len check, or they will be lost
    
    #All word lengths should be >1 character and <= length of the longest word in the English language. It's common for people spam incoherent letters on the Internet.
    text = [word for word in text if word not in en_stopwords and len(word)>1 and len(word) <= len('pneumonoultramicroscopicsilicovolcanoconiosis')] 
    
    clean_text = [num2word(w) for w in text] #just in case any lone numbers appeared after cleaning
    
    return clean_text


def stop_clean(texts):
    texts = [parse_clean(doc) for doc in texts]
    texts = [remove_stopwords(doc) for doc in texts] #just in case, remove stopwords one more time
    texts = [t for tt in texts for t in tt]
            
    return texts

#at this point we will make n-grams, then lemmatise using spacy since it can go by permitted postags

def spacy_lemma(bow,allowed_postags=['NOUN']): #can add any from https://spacy.io/api/annotation#pos-tagging but will be sticking to NOUN, VERB, ADJ
    
    lemma_doc = nlp(" ".join(bow)) 

    lemma_text = [token.text if '_' in token.text else token.lemma_ if token.pos_ in allowed_postags else '' for token in lemma_doc]
    
    return lemma_text

In [46]:
model2_df = final_df[['review_comment']]
model2_df.head()

,review_comment
0,"[Greensboro, to, Las, Vegas, via, Charlotte, I..."
1,"[Not, Verified, Tokyo, to, Dallas, I, was, dre..."
2,"[Providence, to, Tucson, via, Chicago, America..."
3,"[Not, Verified, Philadelphia, to, Boston, Horr..."
4,"[Houston, to, Dallas, I, had, departed, from, ..."


In [47]:
model2_df['review_comment'] = model2_df['review_comment'].map(lambda x: stop_clean(x))
model2_df.head()

C:\Users\Mancef\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,review_comment
0,"[greensboro, las, vegas, charlotte, fly, ameri..."
1,"[verified, tokyo, dallas, dreading, flight, bo..."
2,"[providence, tucson, chicago, american, airlin..."
3,"[verified, philadelphia, boston, horrible, cus..."
4,"[houston, dallas, departed, american, airlines..."


In [48]:
#Now that the cleaner text is ready (only or mostly nouns), we create n-grams again

# Credit to https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for the n-grams code

# Build the bigram and trigram models
bigram = gensim.models.Phrases(list(model2_df['review_comment']), min_count=5, threshold=10) # feed a list of lists of words e.g. [['word1','word2'],['word3','word4'] to get bigrams]
trigram = gensim.models.Phrases(bigram[list(model2_df['review_comment'])], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [49]:
model2_df['3gram_reviews'] = make_trigrams(model2_df['review_comment'])
model2_df['3grams_nouns'] = model2_df['3gram_reviews'].map(lambda x: spacy_lemma(x))
model2_df['3grams_nouns_verbs'] = model2_df['3gram_reviews'].map(lambda x: spacy_lemma(x,allowed_postags=['NOUN','VERB']))
model2_df.head()

C:\Users\Mancef\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,review_comment,3gram_reviews,3grams_nouns,3grams_nouns_verbs
0,"[greensboro, las, vegas, charlotte, fly, ameri...","[greensboro, las_vegas_charlotte, fly_american...","[, las_vegas_charlotte, fly_american, , , time...","[, las_vegas_charlotte, fly_american, fly, , t..."
1,"[verified, tokyo, dallas, dreading, flight, bo...","[verified, tokyo, dallas, dreading, flight, bo...","[, , , , flight, , , , , awful_experience, , f...","[verify, , , dread, flight, book, , , expect, ..."
2,"[providence, tucson, chicago, american, airlin...","[providence, tucson, chicago, american_airline...","[, , , american_airlines, , , carrier, poor_pl...","[, , , american_airlines, unorganize, , carrie..."
3,"[verified, philadelphia, boston, horrible, cus...","[verified_philadelphia, boston, horrible_custo...","[verified_philadelphia, , horrible_customer_se...","[verified_philadelphia, , horrible_customer_se..."
4,"[houston, dallas, departed, american, airlines...","[houston_dallas, departed, american_airlines, ...","[houston_dallas, , american_airlines, time, , ...","[houston_dallas, depart, american_airlines, ti..."


In [50]:
en_stopwords = list(set(STOPWORDS))
en_stopwords.extend(['good','better','great','lot','game','like','I','i','one','two','three']) #after making n-grams, removing numbers should reduce noise

model2_df['3grams_nouns'] = model2_df['3grams_nouns'].map(lambda x: remove_stopwords(x)) #removes blanks as well
model2_df['3grams_nouns_verbs'] = model2_df['3grams_nouns_verbs'].map(lambda x: remove_stopwords(x)) 

In [51]:
model2_df.head()

,review_comment,3gram_reviews,3grams_nouns,3grams_nouns_verbs
0,"[greensboro, las, vegas, charlotte, fly, ameri...","[greensboro, las_vegas_charlotte, fly_american...","[las_vegas_charlotte, fly_american, time, week...","[las_vegas_charlotte, fly_american, fly, time,..."
1,"[verified, tokyo, dallas, dreading, flight, bo...","[verified, tokyo, dallas, dreading, flight, bo...","[flight, awful_experience, flight, demand, boa...","[verify, dread, flight, book, expect, awful_ex..."
2,"[providence, tucson, chicago, american, airlin...","[providence, tucson, chicago, american_airline...","[american_airlines, carrier, poor_planning, cu...","[american_airlines, unorganize, carrier, poor_..."
3,"[verified, philadelphia, boston, horrible, cus...","[verified_philadelphia, boston, horrible_custo...","[verified_philadelphia, horrible_customer_serv...","[verified_philadelphia, horrible_customer_serv..."
4,"[houston, dallas, departed, american, airlines...","[houston_dallas, departed, american_airlines, ...","[houston_dallas, american_airlines, time, gate...","[houston_dallas, depart, american_airlines, ti..."


# LDA Model - 3grams - Nouns Only

In [52]:
#build dictionary and corpus from 3gram dataset, NOUNS only with filter_extremes()

documents = list(model2_df['3grams_nouns'])
dictionary = gensim.corpora.Dictionary(documents)
dictionary.filter_extremes(no_below = 5,no_above=0.5) #trying with default settings
corpus = [dictionary.doc2bow(word) for word in documents]

In [53]:
# LDA model parameters -- we will go with just 5 topics to keep the model more generalised
num_topics = 5
passes = 100
eval_every = None #Evaluation will happen later so no need to evaluate while training

In [54]:
#NOTE: LDAMultiCore; set workers = n-1 (where n is your number of cores)

%time ldamodel1 = LdaMulticore(corpus, num_topics=num_topics, id2word = dictionary, passes=passes, alpha='asymmetric',eval_every=eval_every,workers=3)

# Check resulting topics.
topic_list = ldamodel1.print_topics(num_topics=num_topics, num_words=15)
for index, i in enumerate(topic_list):
    str1 = str(i[1])
    for c in "0123456789+*\".":
        str1 = str1.replace(c, "")
    str1 = str1.replace("  ", " ")
    print(str1)

Wall time: 5min 10s
time seat passenger plane staff aircraft ryanair crew service cabin_crew people return problem airline food
time airport plane airline american_airlines day ticket hour customer_service gate delay bag people staff check
seat service food business_class time economy emirate crew experience meal airline passenger lounge cabin staff
service meal food crew time breakfast drink choice british_airways economy cabin_crew class snack club_europe passenger
seat american_airlines bag plane airline family child people passenger staff customer flight_attendant way baby husband


In [56]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel1.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda1 = CoherenceModel(model=ldamodel1, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_lda1 = coherence_model_lda1.get_coherence()
print('\nCoherence Score: ', coherence_lda1)


Perplexity:  -7.1477789452377

Coherence Score:  0.49110634574817863


In [58]:
pyLDAvis.enable_notebook()
vis = p_gensim.prepare(ldamodel1, corpus, dictionary)
pyLDAvis.save_html(vis, '../supercase-airplane/viz/lda_03_raw.html')
vis

C:\Users\Mancef\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.170444 -0.141038       1        1  33.426865
0      0.007841 -0.057184       2        1  24.081749
2     -0.148828  0.086114       3        1  23.479254
3     -0.215132 -0.041138       4        1  10.046204
4      0.185674  0.153245       5        1   8.965925, topic_info=     Category         Freq               Term        Total  loglift  logprob
94    Default  6309.000000               seat  6309.000000  30.0000  30.0000
42    Default  2068.000000  american_airlines  2068.000000  29.0000  29.0000
687   Default  1650.000000     business_class  1650.000000  28.0000  28.0000
130   Default  4168.000000            service  4168.000000  27.0000  27.0000
138   Default  2902.000000               food  2902.000000  26.0000  26.0000
382   Default  1378.000000               meal  1378.000000  25.0000  25.0000
377   Default  2328.000000               crew  2328.000000  24.0000  24.0000
276   Default  1777.000000                bag  1777.000000  23.0000  23.0000
90    Default   579.000000             family   579.000000  22.0000  22.0000
21    Default  3906.000000              plane  3906.000000  21.0000  21.0000
34    Default  1179.000000            economy  1179.000000  20.0000  20.0000
4     Default  2244.000000            airport  2244.000000  19.0000  19.0000
971   Default  1451.000000         cabin_crew  1451.000000  18.0000  18.0000
321   Default  1449.000000           aircraft  1449.000000  17.0000  17.0000
3524  Default  1384.000000            ryanair  1384.000000  16.0000  16.0000
970   Default   560.000000          breakfast   560.000000  15.0000  15.0000
45    Default  1190.000000   customer_service  1190.000000  14.0000  14.0000
962   Default   777.000000             lounge   777.000000  13.0000  13.0000
633   Default   347.000000              child   347.000000  12.0000  12.0000
142   Default  1809.000000             people  1809.000000  11.0000  11.0000
691   Default   643.000000              drink   643.000000  10.0000  10.0000
635   Default   941.000000              class   941.000000   9.0000   9.0000
2745  Default  1069.000000            emirate  1069.000000   8.0000   8.0000
605   Default   888.000000             choice   888.000000   7.0000   7.0000
125   Default  3172.000000          passenger  3172.000000   6.0000   6.0000
121   Default  1168.000000            luggage  1168.000000   5.0000   5.0000
3     Default  3315.000000            airline  3315.000000   4.0000   4.0000
924   Default  1474.000000    british_airways  1474.000000   3.0000   3.0000
32    Default  1009.000000           customer  1009.000000   2.0000   2.0000
84    Default   574.000000              hotel   574.000000   1.0000   1.0000
...       ...          ...                ...          ...      ...      ...
209    Topic5    80.777542               rude   117.042526   2.0409  -5.7233
94     Topic5  1932.970947               seat  6309.450195   1.2287  -2.5481
3220   Topic5    71.834557             weight   116.143005   1.9313  -5.8406
2010   Topic5    49.622318           two_bags    71.007858   2.0534  -6.2105
276    Topic5   498.799408                bag  1777.731812   1.1408  -3.9028
42     Topic5   554.267761  american_airlines  2068.841064   1.0946  -3.7973
652    Topic5    98.454193              group   212.357254   1.6431  -5.5254
437    Topic5    85.776497              woman   175.704468   1.6947  -5.6632
979    Topic5    95.632919          pay_extra   208.674255   1.6315  -5.5544
213    Topic5    88.467964           suitcase   192.038879   1.6367  -5.6323
91     Topic5   165.183960  flight_attendants   536.748657   1.2333  -5.0079
32     Topic5   240.203262           customer  1009.315002   0.9762  -4.6335
142    Topic5   332.213684             people  1809.253662   0.7168  -4.3092
3      Topic5   432.795441            airline  3315.323486   0.3757  -4.0447
463    Topic5   164.756363           

In [59]:
#save the model

newpath = '../supercase-airplane/lda' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
ldamodel1.save('../supercase-airplane/lda/model1.model')

#Usable model with good topics! Very similar to topics found in first run and arguably better!

### Preliminary Review

This model looks much more usable than previous ones.

Coherence score is higher as well. I'm starting to see some of the coherent topics I previously identified as well.

We can further refine the nouns-only model by removing more stopwords.

Changes to the approach:

1. Cleaning
    - Removed everything other than Nouns and n-grams
    - Used filter_extremes() with default settings (no_below = 5, no_above = 0.5)
   
2. Modelling
    - Decided to use 5 topics only, because I am only looking for 5 general topics (for potential use in a later model that can generalise across the series and potentially the whole strategy genre... or at least the Total War series)
    
    
Changes to Results:
1. 1.5x the Coherence score of previous models (~0.53 coherence! Previous models had ~ 0.38 coherence even with ~10 topics!)
2. 5 topics that seem very coherent

# Fine-tuning

Attempting to refine the results by removing more terms that may be generating noise

In [60]:
en_stopwords = list(set(STOPWORDS))
en_stopwords.extend(['good','better','great','lot','game','like','I','i','one','two','three','thing','bit','total_war','time','10_10','love','fun','play','hour']) #after making bigrams, removing numbers should reduce noise
model2_df['3grams_nouns_v2'] = model2_df['3grams_nouns'].map(lambda x: remove_stopwords(x))

In [61]:
#build dictionary and corpus from 3gram dataset -- this time with filter_extremes

documents3 = list(model2_df['3grams_nouns_v2'])
dictionary3 = gensim.corpora.Dictionary(documents3)
dictionary3.filter_extremes(no_below = 5,no_above=0.5)
corpus3 = [dictionary3.doc2bow(word) for word in documents3]

In [62]:
# LDA model parameters.
num_topics = 5
passes = 100
eval_every = None #Evaluation will happen later so no need to evaluate while training

In [63]:
#Beware, LDA has some randomness to it if you do not set a random_state. May not produce high-coherence model every time.

%time ldamodel3 = LdaMulticore(corpus3, num_topics=num_topics, id2word = dictionary3, passes=passes, alpha='asymmetric',eval_every=eval_every,workers=3,random_state = 14180)

# Check resulting topics.
topic_list = ldamodel3.print_topics(num_topics=num_topics, num_words=15)
for index, i in enumerate(topic_list):
    str1 = str(i[1])
    for c in "0123456789+*\".":
        str1 = str1.replace(c, "")
    str1 = str1.replace("  ", " ")
    print(str1)

Wall time: 4min 50s
seat food service business_class staff airline economy passenger plane experience meal emirate british_airways cabin_crew aircraft
ryanair bag seat luggage staff check people airline passenger airport plane problem way pay return
plane american_airlines airport gate delay passenger airline people staff hotel day connecting_flight problem connection minute
crew service passenger cabin_crew food lounge meal breakfast drink class cabin aircraft british_airways choice seat
ticket airline customer_service day american_airlines customer seat airport change refund phone company email british_airways agent


In [64]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel3.log_perplexity(corpus3))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda3 = CoherenceModel(model=ldamodel3, texts=documents3, dictionary=dictionary3, coherence='c_v')
coherence_lda3 = coherence_model_lda3.get_coherence()
print('\nCoherence Score: ', coherence_lda3)


Perplexity:  -7.188895847682509

Coherence Score:  0.5229892927243615


In [65]:
pyLDAvis.enable_notebook()
vis = p_gensim.prepare(ldamodel3, corpus3, dictionary3)
pyLDAvis.save_html(vis, '../supercase-airplane/viz/lda_04_raw.html')
vis

C:\Users\Mancef\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.161187 -0.030671       1        1  32.291176
2      0.127175  0.141845       2        1  21.619003
1      0.066704 -0.171163       3        1  16.025286
3     -0.221375  0.055808       4        1  15.387837
4      0.188683  0.004181       5        1  14.676693, topic_info=     Category         Freq               Term        Total  loglift  logprob
3517  Default  1270.000000            ryanair  1270.000000  30.0000  30.0000
373   Default  2381.000000               crew  2381.000000  29.0000  29.0000
135   Default  2951.000000               food  2951.000000  28.0000  28.0000
92    Default  6296.000000               seat  6296.000000  27.0000  27.0000
41    Default  2095.000000  american_airlines  2095.000000  26.0000  26.0000
272   Default  1668.000000                bag  1668.000000  25.0000  25.0000
21    Default  3982.000000              plane  3982.000000  24.0000  24.0000
94    Default  1445.000000             ticket  1445.000000  23.0000  23.0000
683   Default  1688.000000     business_class  1688.000000  22.0000  22.0000
64    Default  1296.000000               gate  1296.000000  21.0000  21.0000
44    Default  1184.000000   customer_service  1184.000000  20.0000  20.0000
378   Default  1392.000000               meal  1392.000000  19.0000  19.0000
118   Default  1108.000000            luggage  1108.000000  18.0000  18.0000
58    Default  1099.000000              delay  1099.000000  17.0000  17.0000
4     Default  2217.000000            airport  2217.000000  16.0000  16.0000
85    Default  1520.000000                day  1520.000000  15.0000  15.0000
965   Default  1466.000000         cabin_crew  1466.000000  14.0000  14.0000
956   Default   794.000000             lounge   794.000000  13.0000  13.0000
33    Default  1197.000000            economy  1197.000000  12.0000  12.0000
127   Default  4200.000000            service  4200.000000  11.0000  11.0000
964   Default   566.000000          breakfast   566.000000  10.0000  10.0000
687   Default   651.000000              drink   651.000000   9.0000   9.0000
918   Default  1494.000000    british_airways  1494.000000   8.0000   8.0000
3     Default  3283.000000            airline  3283.000000   7.0000   7.0000
122   Default  3201.000000          passenger  3201.000000   6.0000   6.0000
751   Default   696.000000      entertainment   696.000000   5.0000   5.0000
191   Default  1194.000000              check  1194.000000   4.0000   4.0000
29    Default  1271.000000              cabin  1271.000000   3.0000   3.0000
82    Default   584.000000              hotel   584.000000   2.0000   2.0000
593   Default   347.000000             refund   347.000000   1.0000   1.0000
...       ...          ...                ...          ...      ...      ...
343    Topic5   317.262360              agent   555.094971   1.3595  -4.8124
85     Topic5   722.783142                day  1520.057617   1.1755  -3.9890
31     Topic5   494.877106           customer   998.732178   1.2167  -4.3678
43     Topic5   347.275299             change   659.141724   1.2781  -4.7220
217    Topic5   166.340118            booking   280.191223   1.3975  -5.4581
302    Topic5   109.019676             answer   163.883209   1.5113  -5.8806
340    Topic5   254.296173              money   523.876221   1.1962  -5.0336
3      Topic5   843.834534            airline  3283.102051   0.5603  -3.8342
41     Topic5   604.005188  american_airlines  2095.848877   0.6748  -4.1685
765    Topic5   171.279221                fee   328.879913   1.2665  -5.4288
18     Topic5   123.886612               mile   207.597015   1.4027  -5.7528
52     Topic5   271.864136               help   698.453735   0.9753  -4.9668
361    Topic5   201.155792             number   448.601685   1.1169  -5.2681
261    Topic5   195.893738               book   465.703827   1.0529  -5.2946
98     Topic5   128.277649       comp

In [66]:
newpath = '../supercase-airplane/lda' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
ldamodel3.save('../supercase-airplane/lda/model3.model')

# Randomness in LDA

Due to the randomness in LDA, topics identified are likely to differ each time it is run.

For consistent results, best to include random_state when training the model. (Will do this for future iterations)

The next cell will load a pre-trained model that was used for the remainder of the project.

It has the topics that are general and coherent, and which best fit the purpose of this project.

### This Model

Topics are reasonably coherent, but more suitable for another purpose.

1. Topic 1 - Game Features/Strategic Gameplay (Multiple topics)
2. Topic 2 - Series Authenticity
3. Topic 3 - Tactical Gameplay
4. Topic 4 - Characters
5. Topic 5 - Downloadable Content and Patches

In [143]:
#set up the saved model

lda_saved = LdaMulticore.load('./models/nouns_only/saved_model/saved.model')

documents_saved = list(model2_df['3grams_nouns_v2'])

dictionary_saved = gensim.corpora.Dictionary.load('./models/nouns_only/saved_model/saved.model.id2word')

#dictionary_saved.filter_extremes(no_below = 5,no_above=0.5) #default settings were used when training this model

corpus_saved = [dictionary_saved.doc2bow(doc) for doc in documents_saved]

C:\Users\Alfred\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [145]:
pyLDAvis.enable_notebook()
vis = p_gensim.prepare(lda_saved, corpus_saved, dictionary_saved)
vis

C:\Users\Alfred\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.079664  0.096287       1        1  34.039379
0      0.131353  0.168991       2        1  33.004292
1     -0.207955 -0.074703       3        1  13.383114
2     -0.036563 -0.029150       4        1  11.119786
4      0.192829 -0.161426       5        1   8.453429, topic_info=     Category        Freq            Term       Total  loglift  logprob
71    Default  754.000000            unit  754.000000  30.0000  30.0000
14    Default  614.000000         faction  614.000000  29.0000  29.0000
0     Default  532.000000            army  532.000000  28.0000  28.0000
63    Default  228.000000            hour  228.000000  27.0000  27.0000
245   Default  296.000000             fan  296.000000  26.0000  26.0000
217   Default  210.000000           china  210.000000  25.0000  25.0000
46    Default  195.000000             war  195.000000  24.0000  24.0000
254   Default  146.000000           issue  146.000000  23.0000  23.0000
138   Default  247.000000  best_total_war  247.000000  22.0000  22.0000
174   Default  615.000000        campaign  615.000000  21.0000  21.0000
12    Default  583.000000       diplomacy  583.000000  20.0000  20.0000
6     Default  143.000000         cao_cao  143.000000  19.0000  19.0000
48    Default  111.000000           lu_bu  111.000000  18.0000  18.0000
116   Default  354.000000          series  354.000000  17.0000  17.0000
325   Default  127.000000             bug  127.000000  16.0000  16.0000
850   Default  136.000000          review  136.000000  15.0000  15.0000
295   Default   88.000000         liu_bei   88.000000  14.0000  14.0000
58    Default  895.000000          battle  895.000000  13.0000  13.0000
478   Default   90.000000       yuan_shao   90.000000  12.0000  12.0000
326   Default  145.000000         release  145.000000  11.0000  11.0000
29    Default  135.000000            rome  135.000000  10.0000  10.0000
554   Default  118.000000          vassal  118.000000   9.0000   9.0000
111   Default  217.000000          shogun  217.000000   8.0000   8.0000
738   Default   61.000000     multiplayer   61.000000   7.0000   7.0000
331   Default   69.000000           crash   69.000000   6.0000   6.0000
65    Default  293.000000            love  293.000000   5.0000   5.0000
27    Default  300.000000            play  300.000000   4.0000   4.0000
5     Default  170.000000        building  170.000000   3.0000   3.0000
253   Default  175.000000         history  175.000000   2.0000   2.0000
69    Default  416.000000  three_kingdoms  416.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
1450   Topic5    6.004202  balance_issues    7.181678   2.2915  -6.3717
198    Topic5   44.516201           patch   66.203148   2.0737  -4.3683
325    Topic5   68.842430             bug  127.503433   1.8543  -3.9324
63     Topic5  107.331467            hour  228.979370   1.7129  -3.4883
1239   Topic5   10.719231           today   14.544914   2.1654  -5.7922
530    Topic5   16.173019            sega   25.064259   2.0325  -5.3809
850    Topic5   59.562275          review  136.630066   1.6403  -4.0772
930    Topic5   21.856722           steam   38.040504   1.9165  -5.0797
324    Topic5   19.195147       blood_dlc   31.845112   1.9644  -5.2095
398    Topic5   18.649656            week   30.985853   1.9629  -5.2384
168    Topic5   30.450336          update   60.959038   1.7765  -4.7481
468    Topic5   17.353306       community   28.347240   1.9799  -5.3104
29     Topic5   49.877270            rome  135.936951   1.4680  -4.2546
171    Topic5   43.644951            work  128.750916   1.3888  -4.3881
724    Topic5   24.532148             art   52.307388   1.7134  -4.9642
174    Topic5  111.022919        campaign  615.407227   0.7581  -3.4545
998    Topic5   19.609394          minute   36.627354   1.8458  -5.1882
326    Topic5   43.238186         release  145.1075

# Topics Identified

The nouns-only iterations have clear topic separation between all topics, and the saved one from a previous iteration has coherent topics.

1. **Content & Authenticity**</br>
    - (Topic 2) Game content and its authenticity to the Total War series
    - Contains comparisons to other titles in the Total War series (e.g. shogun_two --> Shogun 2, one of the most well-received installments in the series)
    - Important because target market is full of hardcore history/ROTK buffs 
    - Total War has also carved a nice niche for itself in the game industry - the only closest competitor is Ultimate General, created by a Total War modder </br> </br>

2. **Strategic Gameplay**</br>
    - (Topic 4) Gameplay on the strategic scale, e.g. movements on the campaign map, city management, diplomacy, negotiations
    - Inclusive of new features e.g. revamped Diplomacy, Romance/Records mode selection, Faction playstyles </br> </br>

3. **Tactical Gameplay**</br>
    - (Topic 1) Gameplay of individual battles, managing generals' equipment and skills, their retinues, individual units, managing individual cities, etc </br> </br>

4. **Characters**</br>
    - (Topic 3) Characters in the game and their behaviours.
    - Game is based on both Records and Romance of the Three Kingdoms, historical records (Chen Shou, ~300 AD) and a novel (Luo Guanzhong, 14th Century AD) respectively. 
    - Strong overlap with Topic 1.
    - Draws comparisons to Koei Tecmo's Dynasty Warriors series based on the same source material because of Romance Mode, where generals are lone units capable of incredible feats in battle. </br> </br>

5. **UI/UX, Performance, Stability**</br>
    - (Topic 5) Bugs, crashes, and fixes are terms relevant to this topic and they often come with games that work on massive scales like Total War.
    - General performance of the game (each player uses different PC specifications)
    - Bugs are always a concern and are virtually unavoidable for games, especially close to release.
    - Examples include bugs affecting random crashes in the middle of the game and [crashes involving Liu Bei's annexation ability when the game was first released](https://steamcommunity.com/app/779340/discussions/0/1642038749328500806/).

The corrections made from previous versions of the model seem to have had a tremendously positive effect on getting coherent topics out of this model.

In [216]:
#save visualisation of LDA model
LDA_fin = pyLDAvis.gensim.prepare(lda_saved, corpus_saved, dictionary_saved)
pyLDAvis.save_html(LDA_fin, './viz/lda_fin.html')

C:\Users\Alfred\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [203]:
# Trying out nouns and verbs
en_stopwords = list(set(STOPWORDS))
en_stopwords.extend(['good','better','great','lot','game','like','I','i','one','two','three','thing','bit','total_war','time','10_10','love','fun','play','hour']) 
model2_df['3grams_nouns_verbs_v2'] = model2_df['3grams_nouns_verbs'].map(lambda x: remove_stopwords(x))

In [204]:
#build dictionary and corpus from 3gram dataset -- this time with filter_extremes

documents4 = list(model2_df['3grams_nouns_verbs_v2'])
dictionary4 = gensim.corpora.Dictionary(documents4)
dictionary4.filter_extremes(no_below = 5,no_above=0.5)
corpus4 = [dictionary4.doc2bow(word) for word in documents4]

In [205]:
# LDA model parameters.
num_topics = 5
passes = 100
eval_every = None #Evaluation will happen later so no need to evaluate while training

In [206]:
#Beware, LDA has some randomness to it if you do not set a random_state. May not produce high-coherence model every time.

seed = np.random.randint(0,999999)
print("Seed:", seed,"\n")
%time ldamodel4 = LdaMulticore(corpus4, num_topics=num_topics, id2word = dictionary4, passes=passes, alpha='asymmetric',eval_every=eval_every,workers=3,random_state = seed)

# Check resulting topics.
topic_list = ldamodel4.print_topics(num_topics=num_topics, num_words=15)
for index, i in enumerate(topic_list):
    str1 = str(i[1])
    for c in "0123456789+*\".":
        str1 = str1.replace(c, "")
    str1 = str1.replace("  ", " ")
    print(str1)

Seed: 341010 

Wall time: 1min 49s
series feel diplomacy three_kingdoms best_total_war campaign fan shogun_two battle think release total_war_games buy title tw
battle character general campaign ca add look three_kingdoms unit duel hero want feel china romance_mode
battle diplomacy turn blood friend delay feel hope work character month support dlc change reason
turn war crash cao_cao lu_bu fix start fight liu_bei army yuan_shao campaign know want faction
unit faction army ai battle diplomacy general character building need use campaign enemy build feel


In [207]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel4.log_perplexity(corpus4))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda4 = CoherenceModel(model=ldamodel4, texts=documents4, dictionary=dictionary4, coherence='c_v')
coherence_lda4 = coherence_model_lda4.get_coherence()
print('\nCoherence Score: ', coherence_lda3)


Perplexity:  -6.802324161975279

Coherence Score:  0.5454689780490602


In [208]:
pyLDAvis.enable_notebook()
vis = p_gensim.prepare(ldamodel4, corpus4, dictionary4)
vis

C:\Users\Alfred\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.202884  0.026048       1        1  30.848213
4     -0.057505 -0.162022       2        1  25.479015
1      0.068379 -0.073612       3        1  21.897511
3     -0.198566  0.034816       4        1  15.133147
2     -0.015191  0.174771       5        1   6.642116, topic_info=     Category        Freq            Term       Total  loglift  logprob
108   Default  731.000000            unit  731.000000  30.0000  30.0000
163   Default  553.000000          series  553.000000  29.0000  29.0000
193   Default  289.000000  best_total_war  289.000000  28.0000  28.0000
19    Default  653.000000       diplomacy  653.000000  27.0000  27.0000
1     Default  521.000000            army  521.000000  26.0000  26.0000
70    Default  309.000000            turn  309.000000  25.0000  25.0000
23    Default  613.000000         faction  613.000000  24.0000  24.0000
9     Default  339.000000              ca  339.000000  23.0000  23.0000
334   Default  340.000000             fan  340.000000  22.0000  22.0000
7     Default  163.000000        building  163.000000  21.0000  21.0000
74    Default  202.000000             war  202.000000  20.0000  20.0000
94    Default  955.000000          battle  955.000000  19.0000  19.0000
117   Default  442.000000         general  442.000000  18.0000  18.0000
457   Default  133.000000           crash  133.000000  17.0000  17.0000
105   Default  326.000000      shogun_two  326.000000  16.0000  16.0000
99    Default  247.000000            hero  247.000000  15.0000  15.0000
10    Default  149.000000         cao_cao  149.000000  14.0000  14.0000
77    Default  119.000000           lu_bu  119.000000  13.0000  13.0000
165   Default  195.000000              tw  195.000000  12.0000  12.0000
96    Default  623.000000       character  623.000000  11.0000  11.0000
148   Default  222.000000            duel  222.000000  10.0000  10.0000
106   Default  492.000000  three_kingdoms  492.000000   9.0000   9.0000
146   Default  284.000000             add  284.000000   8.0000   8.0000
405   Default   96.000000         liu_bei   96.000000   7.0000   7.0000
281   Default  224.000000           fight  224.000000   6.0000   6.0000
654   Default   95.000000       yuan_shao   95.000000   5.0000   5.0000
83    Default  658.000000            feel  658.000000   4.0000   4.0000
325   Default  220.000000           china  220.000000   3.0000   3.0000
249   Default  123.000000            mode  123.000000   2.0000   2.0000
25    Default  157.000000             fix  157.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
104    Topic5   22.066025          return   51.397076   1.8662  -5.2129
360    Topic5   28.864130          friend   93.338356   1.5381  -4.9443
703    Topic5   21.327688          screen   56.177246   1.7432  -5.2469
47     Topic5   24.985891          reason   84.523926   1.4930  -5.0886
437    Topic5   23.110836         warlord   73.077667   1.5605  -5.1666
19     Topic5   61.308674       diplomacy  653.037903   0.3460  -4.1910
126    Topic5   25.694761             dlc  109.243027   1.2645  -5.0606
1125   Topic5   12.065248       kong_rong   21.513641   2.1334  -5.8166
128    Topic5   26.599817            hope  126.130394   1.1553  -5.0260
94     Topic5   62.112701          battle  955.781738  -0.0218  -4.1780
70     Topic5   36.436089            turn  309.514191   0.5723  -4.7114
669    Topic5   21.931709         setting   90.003525   1.2998  -5.2190
380    Topic5   16.251379         capture   44.814949   1.6974  -5.5187
706    Topic5   17.102768          expand   54.557831   1.5517  -5.4677
55     Topic5   16.934690            shit   53.253616   1.5660  -5.4776
393    Topic5   20.926258           force  104.780441   1.1009  -5.2659
40     Topic5   21.285732            mean  115.742661   1.0184  -5.2489
125    Topic5   16.449766       developer   49.5398

In [209]:
# LDA model parameters.
num_topics = 7
passes = 100
eval_every = None #Evaluation will happen later so no need to evaluate while training

In [210]:
#Beware, LDA has some randomness to it if you do not set a random_state. May not produce high-coherence model every time.

seed = np.random.randint(0,999999)
print("Seed:", seed,"\n")
%time ldamodel4 = LdaMulticore(corpus4, num_topics=num_topics, id2word = dictionary4, passes=passes, alpha='asymmetric',eval_every=eval_every,workers=3,random_state = seed)

# Check resulting topics.
topic_list = ldamodel4.print_topics(num_topics=num_topics, num_words=15)
for index, i in enumerate(topic_list):
    str1 = str(i[1])
    for c in "0123456789+*\".":
        str1 = str1.replace(c, "")
    str1 = str1.replace("  ", " ")
    print(str1)

Seed: 736409 

Wall time: 1min 58s
feel three_kingdoms best_total_war campaign diplomacy series character release battle work change ca think shogun_two add
series fan battle shogun_two three_kingdoms total_war_games enjoy ca feel title dynasty_warriors romance_mode diplomacy campaign one_best
unit battle faction army character general ai diplomacy campaign feel need building enemy look start
fix crash campaign bug battle tw update issue wait patch release buy review launch franchise
diplomacy lu_bu campaign strategy mechanic battle experience turn friend coalition execute one_turn army want need
buy need look think chinese_history pay dlc win fan blood battle guy s war dlcs
war china cao_cao turn yuan_shao liu_bei vassal army fight want sun_jian kill lu_bu people let


In [211]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel4.log_perplexity(corpus4))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda4 = CoherenceModel(model=ldamodel4, texts=documents4, dictionary=dictionary4, coherence='c_v')
coherence_lda4 = coherence_model_lda4.get_coherence()
print('\nCoherence Score: ', coherence_lda3)


Perplexity:  -6.843546324823157

Coherence Score:  0.5454689780490602


In [212]:
pyLDAvis.enable_notebook()
vis = p_gensim.prepare(ldamodel4, corpus4, dictionary4)
vis

C:\Users\Alfred\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.049111 -0.041127       1        1  38.028172
0      0.105636 -0.016910       2        1  24.371471
1      0.183722 -0.009938       3        1  16.643620
6     -0.289007  0.001594       4        1   6.459582
3      0.077198 -0.102770       5        1   6.339120
4     -0.049134 -0.092879       6        1   5.198752
5      0.020696  0.262030       7        1   2.959291, topic_info=     Category        Freq            Term       Total  loglift  logprob
1     Default  522.000000            army  522.000000  30.0000  30.0000
8     Default  296.000000             buy  296.000000  29.0000  29.0000
19    Default  655.000000       diplomacy  655.000000  28.0000  28.0000
94    Default  955.000000          battle  955.000000  27.0000  27.0000
241   Default  707.000000        campaign  707.000000  26.0000  26.0000
334   Default  340.000000             fan  340.000000  25.0000  25.0000
25    Default  156.000000             fix  156.000000  24.0000  24.0000
163   Default  557.000000          series  557.000000  23.0000  23.0000
101   Default  361.000000            need  361.000000  22.0000  22.0000
108   Default  733.000000            unit  733.000000  21.0000  21.0000
70    Default  308.000000            turn  308.000000  20.0000  20.0000
74    Default  200.000000             war  200.000000  19.0000  19.0000
140   Default  346.000000           think  346.000000  18.0000  18.0000
325   Default  218.000000           china  218.000000  17.0000  17.0000
457   Default  133.000000           crash  133.000000  16.0000  16.0000
179   Default  375.000000            look  375.000000  15.0000  15.0000
449   Default  147.000000             bug  147.000000  14.0000  14.0000
10    Default  147.000000         cao_cao  147.000000  13.0000  13.0000
165   Default  196.000000              tw  196.000000  12.0000  12.0000
235   Default  266.000000        mechanic  266.000000  11.0000  11.0000
188   Default  180.000000        strategy  180.000000  10.0000  10.0000
281   Default  224.000000           fight  224.000000   9.0000   9.0000
105   Default  327.000000      shogun_two  327.000000   8.0000   8.0000
438   Default  131.000000             win  131.000000   7.0000   7.0000
193   Default  293.000000  best_total_war  293.000000   6.0000   6.0000
77    Default  117.000000           lu_bu  117.000000   5.0000   5.0000
654   Default   93.000000       yuan_shao   93.000000   4.0000   4.0000
106   Default  494.000000  three_kingdoms  494.000000   3.0000   3.0000
756   Default  118.000000          vassal  118.000000   2.0000   2.0000
405   Default   94.000000         liu_bei   94.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
1223   Topic7    2.598572        bullshit    5.635776   2.7460  -6.5435
717    Topic7   31.396099             pay   68.114258   2.7457  -4.0518
511    Topic7   25.444328           blood   59.798439   2.6657  -4.2619
1485   Topic7    9.268458            walk   21.025633   2.7011  -5.2718
295    Topic7   22.484728               s   63.543243   2.4813  -4.3856
1380   Topic7    6.190183           water   14.176042   2.6916  -5.6755
30     Topic7   22.791569             guy   72.992477   2.3563  -4.3720
126    Topic7   30.476038             dlc  107.518379   2.2595  -4.0815
8      Topic7   66.755722             buy  296.086060   2.0306  -3.2974
438    Topic7   27.131603             win  131.838776   1.9393  -4.1977
243    Topic7   10.275864         country   33.509979   2.3382  -5.1686
101    Topic7   38.415779            need  361.176788   1.2793  -3.8500
134    Topic7   10.940601        purchase   38.304901   2.2671  -5.1060
1441   Topic7    8.832839          invest   25.928135   2.4434  -5.3200
140    Topic7   32.834778           think  346.791718   1.1630  -4.0070
179    Topic7   33.295006            look  375.298004   1.0979  -3.9930
299    Topic7  

In [213]:
#saving this model that uses nouns and verbs, although the results are not as good as the saved model
newpath = './models/nouns_verbs/model4' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
ldamodel4.save('./models/nouns_verbs/model4/model4.model')

C:\Users\Alfred\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\Alfred\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [67]:
model2_df.to_csv('../supercase-airplane/data/model2_df.csv')

### References

https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/